## X-Ray Daemon Command

```bash
docker run --rm \
    --attach STDOUT \
    -v ~/.aws/:/root/.aws/:ro \
    --net=host \
    -e AWS_REGION=us-east-2 \
    -p 2000:2000/udp \
    public.ecr.aws/xray/aws-xray-daemon:3.x --help
```

```text
Usage: X-Ray [options]
        -a      --resource-arn  Amazon Resource Name (ARN) of the AWS resource running the daemon.
        -o      --local-mode    Don't check for EC2 instance metadata.
        -m      --buffer-memory Change the amount of memory in MB that buffers can use (minimum 3).
        -n      --region        Send segments to X-Ray service in a specific region.
        -b      --bind          Overrides default UDP address (127.0.0.1:2000).
        -t      --bind-tcp      Overrides default TCP address (127.0.0.1:2000).
        -r      --role-arn      Assume the specified IAM role to upload segments to a different account.
        -c      --config        Load a configuration file from the specified path.
        -f      --log-file      Output logs to the specified file path.
        -l      --log-level     Log level, from most verbose to least: dev, debug, info, warn, error, prod (default).
        -p      --proxy-address Proxy address through which to upload segments.
        -v      --version       Show AWS X-Ray daemon version.
        -h      --help          Show this screen
```

## Start the XRay Daemon

```bash
# export profile as env vars
eval "$(aws configure export-credentials --profile cloud-course --format env)"

# run the xray daemon in local mode i.e. skip checking for EC2 instance metadata
docker run --rm \
    --attach STDOUT \
    --net=host \
    -e AWS_REGION=us-west-2 \
    -e AWS_ACCESS_KEY_ID \
    -e AWS_SECRET_ACCESS_KEY \
    -e AWS_SESSION_TOKEN \
    -p 2000:2000/udp \
    public.ecr.aws/xray/aws-xray-daemon:3.x \
        --local-mode \
        --log-level dev \
        --bind 0.0.0.0:2000 \
        --bind-tcp 0.0.0.0:2000
```

In [35]:
import httpx

from aws_xray_sdk.core import (
    patch_all,
    xray_recorder,
)

# Patch all libraries to be instrumented by AWS X-Ray
# patch_all()

sampling_rules = {
    "version": 1,  # Ensure the version field is set correctly
    "rules": [
        {
            "RuleName": "DefaultRule",
            "Priority": 1,
            "FixedRate": 1.0,
            "ReservoirSize": 100,
            "ServiceName": "*",
            "ServiceType": "*",
            "HTTPMethod": "*",
            "URLPath": "*",
            "Host": "*",
            "ResourceARN": "*",
            "Version": 1,
        },
        {
            "RuleName": "default",
            "FixedRate": 0.05,
            "ReservoirSize": 1
        }
    ],
    "default": {
        "FixedRate": 0.1,
        "ReservoirSize": 1
    }
}

# trace config
xray_recorder.configure(
    service="my-service",
    daemon_address="localhost:2000",
    sampling=True,
    sampling_rules=sampling_rules,
)


TypeError: '<' not supported between instances of 'NoneType' and 'int'

In [26]:
async with xray_recorder.in_segment_async("FastAPI Segment") as segment:
    try:
        print("hi")
    except Exception as e:
        segment.put_annotation("error", str(e))
        raise e
    


hi


In [4]:
segment = xray_recorder.begin_segment("my-segment")

In [5]:
from rich import inspect
inspect(segment)

╭──────────────────── <class 'aws_xray_sdk.core.models.segment.Segment'> ────────────────────╮
│ The compute resources running your application logic send data                             │
│ about their work as segments. A segment provides the resource's name,                      │
│ details about the request, and details about the work done.                                │
│                                                                                            │
│ ╭────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ <aws_xray_sdk.core.models.segment.Segment object at 0x1125f6fd0>                       │ │
│ ╰────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                            │
│ annotations = {}                                                                           │
│         aws = {'xray': {'sdk': 'X-Ray for Python', 'sdk_version': '2.14.0'}}               │
│       cause = {}                                                                           │
│        http = {}                                                                           │
│          id = '646b5d01d49c2567'                                                           │
│ in_progress = True                                                                         │
│    metadata = {}                                                                           │
│        name = 'my-segment'                                                                 │
│   parent_id = None                                                                         │
│ ref_counter = <aws_xray_sdk.core.utils.atomic_counter.AtomicCounter object at 0x107a2d520> │
│     sampled = True                                                                         │
│     service = {'runtime': 'CPython', 'runtime_version': '3.9.6'}                           │
│  start_time = 1722356344.44255                                                             │
│ subsegments = []                                                                           │
│    trace_id = '1-66a91278-b4d7dc23286df1b3399a861d'                                        │
│        user = None                                                                         │
╰────────────────────────────────────────────────────────────────────────────────────────────╯

In [28]:
import traceback

segment_2 = xray_recorder.begin_segment("my-segment-2", parent_id=segment.id)
try:
    1 / 0
except ZeroDivisionError as err:
    stack = traceback.format_exc()
    print(stack)
    segment_2.add_exception(err, stack=stack, remote=False)

can not parse stack trace string, ignore stack field.


Traceback (most recent call last):
  File "/var/folders/f3/pxd2mkvj6cvgxr_d9f2brs7w0000gn/T/ipykernel_11016/4164158313.py", line 5, in <module>
    1 / 0
ZeroDivisionError: division by zero



In [29]:
inspect(segment_2)

╭────────────────────────────── <class 'aws_xray_sdk.core.models.segment.Segment'> ───────────────────────────────╮
│ The compute resources running your application logic send data                                                  │
│ about their work as segments. A segment provides the resource's name,                                           │
│ details about the request, and details about the work done.                                                     │
│                                                                                                                 │
│ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ <aws_xray_sdk.core.models.segment.Segment object at 0x115264040>                                            │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                                 │
│ annotations = {}                                                                                                │
│         aws = {'xray': {'sdk': 'X-Ray for Python', 'sdk_version': '2.14.0'}}                                    │
│       cause = {                                                                                                 │
│                   'exceptions': [<aws_xray_sdk.core.models.throwable.Throwable object at 0x115543d30>],         │
│                   'working_directory':                                                                          │
│               '/Users/eric/repos/python-aws-course/python-on-aws-course/section-4--observability'               │
│               }                                                                                                 │
│       fault = True                                                                                              │
│        http = {}                                                                                                │
│          id = 'a546a1082c2f2512'                                                                                │
│ in_progress = True                                                                                              │
│    metadata = {}                                                                                                │
│        name = 'my-segment-2'                                                                                    │
│   parent_id = '3cc83ba34510e017'                                                                                │
│ ref_counter = <aws_xray_sdk.core.utils.atomic_counter.AtomicCounter object at 0x112d90490>                      │
│     sampled = True                                                                                              │
│     service = {'runtime': 'CPython', 'runtime_version': '3.9.6'}                                                │
│  start_time = 1722358237.038078                                                                                 │
│ subsegments = []                                                                                                │
│    trace_id = '1-66a919dd-ad4408e53518bba71e97fa53'                                                             │
│        user = None                                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [16]:
inspect(xray_recorder.get_trace_entity())

╭──────────────────── <class 'aws_xray_sdk.core.models.segment.Segment'> ────────────────────╮
│ The compute resources running your application logic send data                             │
│ about their work as segments. A segment provides the resource's name,                      │
│ details about the request, and details about the work done.                                │
│                                                                                            │
│ ╭────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ <aws_xray_sdk.core.models.segment.Segment object at 0x112e63970>                       │ │
│ ╰────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                            │
│ annotations = {}                                                                           │
│         aws = {'xray': {'sdk': 'X-Ray for Python', 'sdk_version': '2.14.0'}}               │
│       cause = {}                                                                           │
│        http = {}                                                                           │
│          id = '5d590d33f066a974'                                                           │
│ in_progress = True                                                                         │
│    metadata = {}                                                                           │
│        name = 'my-segment-2'                                                               │
│   parent_id = '646b5d01d49c2567'                                                           │
│ ref_counter = <aws_xray_sdk.core.utils.atomic_counter.AtomicCounter object at 0x112e63d00> │
│     sampled = True                                                                         │
│     service = {'runtime': 'CPython', 'runtime_version': '3.9.6'}                           │
│  start_time = 1722356534.072304                                                            │
│ subsegments = []                                                                           │
│    trace_id = '1-66a91336-e250a888c428bbc1b8dc27df'                                        │
│        user = None                                                                         │
╰────────────────────────────────────────────────────────────────────────────────────────────╯